# Model Training

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
module = os.path.abspath('/home/learner/DLA_project/src/main')
# module = os.path.abspath("C:/Users\\18145\\development\\wesad_experiments\\src\\main")
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager

In [2]:
manager = DataManager()

In [3]:

manager.load_all()


Loading data from S2
	Path=/media/learner/6663-3462/WESAD/S2/S2.pkl
Loading data from S3
	Path=/media/learner/6663-3462/WESAD/S3/S3.pkl
Loading data from S4
	Path=/media/learner/6663-3462/WESAD/S4/S4.pkl
Loading data from S5
	Path=/media/learner/6663-3462/WESAD/S5/S5.pkl
Loading data from S6
	Path=/media/learner/6663-3462/WESAD/S6/S6.pkl
Loading data from S7
	Path=/media/learner/6663-3462/WESAD/S7/S7.pkl
Loading data from S8
	Path=/media/learner/6663-3462/WESAD/S8/S8.pkl
Loading data from S9
	Path=/media/learner/6663-3462/WESAD/S9/S9.pkl
Loading data from S10
	Path=/media/learner/6663-3462/WESAD/S10/S10.pkl
Loading data from S11
	Path=/media/learner/6663-3462/WESAD/S11/S11.pkl


In [4]:
manager.compute_features();
manager.compute_features_stress();

conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6
Considering subject  7
Considering subject  8
Considering subject  9
Considering subject  10
Considering subject  11
conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6
Considering subject  7
Considering subject  8
Considering subject  9
Considering subject  10
Considering subject  11


In [30]:
print("We have", len(manager.SUBJECTS), " subjects")

for feature in manager.FEATURES.keys():
    print("there are ", len(manager.FEATURES[feature]), " values for ", feature)

We have 10  subjects
there are  44461  values for  a_mean
there are  44461  values for  a_std
there are  44461  values for  a_maxx
there are  44461  values for  a_maxy
there are  44461  values for  a_maxz
there are  44461  values for  e_max
there are  44461  values for  e_min
there are  44461  values for  e_mean
there are  44461  values for  e_range
there are  44461  values for  e_std
there are  44461  values for  t_max
there are  44461  values for  t_min
there are  44461  values for  t_mean
there are  44461  values for  t_range
there are  44461  values for  t_std


In [31]:
# Lets go ahead and reshape this data 
X = (   manager.FEATURES['a_mean'], manager.FEATURES['a_std'], manager.FEATURES['a_maxx'], manager.FEATURES['a_maxy'], manager.FEATURES['a_maxz'],\
                  manager.FEATURES['e_max'], manager.FEATURES['e_min'], manager.FEATURES['e_mean'], manager.FEATURES['e_range'], manager.FEATURES['e_std'],\
                  manager.FEATURES['t_max'], manager.FEATURES['t_min'], manager.FEATURES['t_mean'], manager.FEATURES['t_range'], manager.FEATURES['t_std'] )
X = np.reshape(X, (len(manager.FEATURES['a_mean'])  ,15))

y = (   manager.STRESS_FEATURES['a_mean'], manager.STRESS_FEATURES['a_std'], manager.STRESS_FEATURES['a_maxx'], manager.STRESS_FEATURES['a_maxy'], manager.STRESS_FEATURES['a_maxz'],\
                  manager.STRESS_FEATURES['e_max'], manager.STRESS_FEATURES['e_min'], manager.STRESS_FEATURES['e_mean'], manager.STRESS_FEATURES['e_range'], manager.STRESS_FEATURES['e_std'],\
                  manager.STRESS_FEATURES['t_max'], manager.STRESS_FEATURES['t_min'], manager.STRESS_FEATURES['t_mean'], manager.STRESS_FEATURES['t_range'], manager.STRESS_FEATURES['t_std'] )
y = np.reshape(y, (len(manager.STRESS_FEATURES['a_mean'])  ,15))


In [1]:
targets_base = [0] * len(X)
targets_stress = [1] * len(y)

NameError: name 'X' is not defined

In [32]:
import pip

try:
    import keras
except ImportError:
    ! sudo pip3 install keras
try:
    import h5py
except ImportError:
    ! sudo pip3 install h5py
try:
    import ibmiotf
except ImportError:
    ! sudo pip3 install ibmiotf
try:
    import tensorflow
except ImportError:
    ! sudo pip3 install tensorflow

In [33]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow
import keras

It will be better to have our data scaled between 0 and 1, so lets go ahead and create a function to 
normalize the data that way now.

In [9]:
def normalize(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [34]:
from keras.preprocessing import sequence

from keras.models import Sequential

from keras.layers import Dense, Embedding

from keras.layers import LSTM

In [35]:
# Split the data up in train and test sets
from sklearn.model_selection import train_test_split

# since we don't have the same number of samples for x and y, 
# we will drop off some x values for creating the sample
X_train, X_test, y_train, y_test = \
    train_test_split(X[0:len(y)], y, test_size=0.20, random_state=42)

In [36]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(19024, 15)
(4756, 15)
(19024, 15)
(4756, 15)


In [ ]:
t_base_train = [0] * len(X_train)
t_stress_train = [0] * len(y_train)

t_base_test = [0] * len(X_test)
t_stress_test = [0] * len(y_test)

In [24]:
X = [X_train, y_train]
print(np.shape(X))

(2, 9512, 15)


In [27]:
print('Build the LSTM network...')
max_features = 20000

model = Sequential()

model.add(Dense(2, input_dim=2, activation='relu'))

model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))

# https://stackoverflow.com/questions/49623436/valueerror-error-when-checking-target-expected-dense-2-to-have-3-dimensions-b
model.add(Dense(1, activation='sigmoid'))

Build the LSTM network...


In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
dense_3 (Dense)              (None, None, 1)           129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy',

              optimizer='adam',

              metrics=['accuracy'])


In [15]:
print('Training LSTM...')

batch_size = 1 # I think I want to use batch_size = 1



model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,

                            batch_size=batch_size)

Training LSTM...


ValueError: Error when checking target: expected dense_1 to have shape (1,) but got array with shape (15,)

In [ ]:
print('score:', score)

print('accuracy:', acc)

In [ ]:
x_test, y_test))

# score, acc = model.evaluate(x_test, y_test,

#                             batch_size=batch_size)

print('Test score:', score)

print('Test accuracy:', acc)

In [ ]:
print((x_train))

features go across to the right, they have 80 features
time goes down , number of samples

In [ ]:
print(len(x_train[0]))
print(type(x_train))

In [ ]:
#manager.train_model()

# manager.test_model()
